In [ ]:
# 传统神经网络存在的问题:
# 1.权值太多
# 2.速度慢
# Tip:
#     数据量小,网络复杂---过拟合
#     数据量大,网络简单---欠拟合
# Tip:
#     权值 : 数据量: 1:5~30倍

In [ ]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [ ]:
# 一、载入数据
mnist=input_data.read_data_sets('Tensorflow_data',one_hot=True)

batch_size=100
    # 定义每个批次的大小--训练模型的时候一批次100个
n_batch=mnist.train.num_examples // batch_size
    # n_batch---一共有多少个批次(计算数据集总数量 // 批次大小)
    # // 整除   

In [ ]:
# +二、参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean=tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev=tf.sqrt(tf.reduce_mean(tf.square(var-mean)))
        tf.summary.scalar('stddev',stddev) # 标准差
        tf.summary.scalar('max',tf.reduce_max(var)) # 最大值
        tf.summary.scalar('min',tf.reduce_min(var)) # 最小值
        tf.summary.histogram('histogram',var) # 直方图 
        
# 初始化权值
def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
        # 生成一个截断的正太分布
    return tf.Variable(initial)

# 初始化偏置
def bias_variale(shape):
    initial=tf.constant(0.1,shape=shape)
    return tf.Variable(initial)

In [ ]:
# 三、卷积层调用
# 使用的是Tensorflow里的一个库
def conv2d(x,W):
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')
# conv2d---二维的卷积操作
# x [批次,长,宽,黑白/彩色--1/3]

In [ ]:
# 四、池化层调用
def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME')
# kszie[1,x,y,1] 只有中间两个自己调节

In [ ]:
# 五、 卷积,池化
# 定义两个变量
with tf.name_scope('input'):
    x=tf.placeholder(tf.float32,[None,784])
    y=tf.placeholder(tf.float32,[None,10])
    with tf.name_scope('x_image'):
        # 改变x的格式为4D的向量 [批次,长,宽,黑白/彩色--1/3]
        x_image=tf.reshape(x,[-1,28,28,1],name='x_image')

In [ ]:
# 初始化第一层卷积层的权值和偏置值(使用命名空间规整)
with tf.name_scope('Conv1'):
    with tf.name_scope('W_conv1'):
        W_conv1=weight_variable([5,5,1,32]) # 5×5的采样窗口,32个卷积核从1个平面抽取特征
    with tf.name_scope('b_conv1'):
        b_conv1=bias_variale([32]) # 每一个卷积核一个偏置值

In [ ]:
# 第一层卷积,池化
with tf.name_scope('conv2d_1'):
   conv2d_1=conv2d(x_image,W_conv1)+b_conv1 # 二维卷积
with tf.name_scope('relu'):
    h_conv1=tf.nn.relu(conv2d_1) # 激活函数
with tf.name_scope('h_pool1'):
    h_pool1=max_pool_2x2(h_conv1) # 池化


In [ ]:
with tf.name_scope('Conv2'):
    with tf.name_scope('Wconv2'):
        # 初始化第二层卷积层的权值和偏置值
        W_conv2=weight_variable([5,5,32,64]) # 5×5的采样窗口,32个卷积核从1个平面抽取特征
    with tf.name_scope('b_conv2'):
        b_conv2=bias_variale([64]) # 每一个卷积核一个偏置值

In [ ]:
# 第二层卷积,池化
with tf.name_scope('conv2d_2'): # 二维卷积
    conv2d_2=conv2d(h_pool1,W_conv2)+b_conv2
with tf.name_scope('relu'):
    h_conv2=tf.nn.relu(conv2d_2)# 激活函数
with tf.name_scope('h_pool2'):
    h_pool2=max_pool_2x2(h_conv2) # 池化

In [ ]:
# 28*28的图片
# 第一次卷积后还是28*28,
# 第一次池化后变为14*14
# 第二次卷积后14*14
# 第二次池化后7*7
# 得到64张7*7 

In [ ]:
# 七、初始化第一个全连接层的权值
with tf.name_scope('fcl'):
    with tf.name_scope('W_fcl'): 
        W_fc1=weight_variable([7*7*64,1024],name='W_fc1') # 上一层有7*7*64个神经元,全连接层有1024个神经元
    
    with tf.name_scope('b_fcl'):
        b_fc1=bias_variale([1024],name='b_fc1') # 1024个节点
    
    with tf.name_scope('h_pool2_flat'):
        # 把第二层池化层的输出扁平化为1维
        h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64],name='h_pool2_flat')

    with tf.name_scope('wx_plus_b1'):
        wx_plus_b1=tf.matmul(h_pool2_flat,W_fc1)+b_fc1
    with tf.name_scope('relu'):    # 第一个全连接层的输出
        h_fc1=tf.nn.relu(wx_plus_b1) # 激活函数

        # 概率表示
    with tf.name_scope('keep_prob'):
        keep_prob=tf.placeholder(tf.float32,name='keep_prob')
    with tf.name_scope('h_fcl_drop'):
        h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob,name='h_fcl_drop')

In [ ]:
# 初始化第二个全连接层
with tf.name_scope('fc2'):
    with tf.name_scope('W_fc2'): 
        W_fc2=weight_variable([1024,10],name='W_fc2')
    with tf.name_scope('b_fc2'):
        b_fc2=bias_variale([10],name='b_fc2')
    with tf.name_scope('wx_plus_b2'):
        wx_plus_b2=tf.matmul(h_fc1_drop,W_fc2)+b_fc2
    with tf.name_scope('softmax'):
        # 计算输出
        prediction=tf.nn.softmax(wx_plus_b2)


In [ ]:
# 交叉熵代价函数
with tf.name_scope('cross_entropy'):
    cross_entropy=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y,logits=prediction))
    tf.summary.scalar('cross_entropy',cross_entropy)
# 使用AdamOptimizer进行优化---优化器
with tf.name_scope('train'):
    train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

# 求准确率
with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        # 结果存放在一个布尔列表中
        correct_prediction=tf.equal(tf.argmax(prediction,1),tf.argmax(y,1))
    with tf.name_scope('accuracy'):
        # 求准确率
        accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf.float32))
        tf.summary.scalar('accuracy',accuracy)

# 合并所有的summary
merged=tf.summary.merge_all()


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    train_writer=tf.summary.FileWriter('logs/train',sess.graph)
    test_writer=tf.summary.FileWriter('logs/test',sess.graph)
    for i in range(21):
        # 训练模型
        for batch in range(n_batch):
            xx,yy=mnist.train.next_batch(batch_size)
            sess.run(train_step,feed_dict={x:xx,y:yy,keep_prob:0.5})
            # 记录训练集计算的参数
            summary=sess.run(merged,feed_dict={x:xx,y:yy,keep_prob:1.0})
            train_writer.add_summary(summary,i)
            # 记录测试集计算的参数
            xx,yy=mnist.test.next_batch(batch_size)
            summary=sess.run(merged,feed_dict={x:xx,y:yy,keep_prob:1.0})
            test_writer.add_summary(summary,i)
            
            if i%100==0:
                test_acc=sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels,keep_prob:1.0})
                train_acc=sess.run(accuracy,feed_dict={x:mnist.train.images[:10000],y:mnist.train.labels[:10000,keep_prob:1.0]})
                print("值:"+str(i)+",测试集准确率:"+str(test_acc))

